In [ ]:
# import tensorflow v2 - disable eager execution to build explicit graph for deployment
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.compat.v1.disable_eager_execution()

# import other libraries
import pandas as pd

import numpy as np
from combined_estimator import CombinedEstimator
from model_fns import * # all model_fns are using python closures

# import transformers
from tf_transformers.binary_tensor_agg.binary_tensor_agg_op import BinaryTensorAggOp
from tf_transformers.binary_tensor_agg.binary_tensor_op import BinaryTensorOp

from tf_transformers.binary_tensor_agg.binary_tensor_agg_transformer import BinaryTensorAggTransformer
from tf_transformers.array_extraction.array_extractor_op import ArrayExtractorOp
from tf_transformers.array_extraction.array_extractor_transformer import ArrayExtractorTransformer
from tf_transformers.math_binary.math_binary_op import BinaryOperation
from tf_transformers.math_binary.math_binary_transformer import MathBinaryTransformer

### Hyperparameters


In [ ]:
LEARNING_RATE=0.001
BATCH_SIZE = 256
LEAKY_RELU_ALPHA = 0.1

### Variables

In [ ]:
MODEL_DIR="o2d"
SAVE_CHECKPOINT_SECS = 900 # once in 15 minutes
SAVE_SUMMARY_STEPS = 4000 # on 4000 epochs
KEEP_CHECKPOINT_MAX = 5 # keep small number to not explode disk space

### Features

In [ ]:
features_fm_regressor = ['banner_factor','weekday','hour','fm_avg_Rz_1H','fm_avg_Rid_1Day','fm_avg_Rid_hour_30D','rest_zone_rain_mode__2H','fm_avg_Rz_dow_hour_7D','fm_avg_Rz_3H','fm_avg_Rz_dow_hour_30D','fm_avg_Rz_3D','fm_avg_Rz_1D','fm_avg_Rh_1D','fm_avg_Rh_hour_3D','fm_avg_Rh_dow_hour_7D']
features_lm_regressor  = ['restaurant_customer_distance','p2d_avg_Rz_3D','p2d_avg_Rz_1H','p2d_avg_Cz_3D','p2d_avg_Rid_20D','p2d_avg_Rid_1H','p2d_avg_Cz_1H','p2d_avg_Rid_3D','r2d_avg_Cg_7D','p2d_avg_Ch_20D','p2d_sd_Ch_20D','rest_zone_rain_mode__2H','cust_zone_rain_mode__2H','embedding_dayofweek_LM_0','embedding_dayofweek_LM_1','embedding_dayofweek_LM_2','embedding_hour_LM_0','embedding_hour_LM_1','embedding_hour_LM_2','embedding_hour_LM_3','embedding_hour_LM_4','embedding_city_LM_0','embedding_city_LM_1','embedding_city_LM_2','embedding_city_LM_3','embedding_city_LM_4','embedding_Rz_LM_0','embedding_Rz_LM_1','embedding_Rz_LM_2','embedding_Rz_LM_3','embedding_Rz_LM_4','embedding_Rz_LM_5','embedding_Rz_LM_6','embedding_Rz_LM_7','embedding_Rz_LM_8','embedding_Rz_LM_9','embedding_Rz_LM_10','embedding_Rz_LM_11','embedding_Rz_LM_12','embedding_Rz_LM_13','embedding_Rz_LM_14','embedding_Rz_LM_15','embedding_Rz_LM_16','embedding_Rz_LM_17','embedding_Rz_LM_18','embedding_Rz_LM_19','embedding_Rz_LM_20','embedding_Rz_LM_21','embedding_Rz_LM_22','embedding_Rz_LM_23','embedding_Rz_LM_24']
features_o2a_regressor = ['banner_factor','weekday','hour','o2as_avg_res_1h','o2a_zoneid_last15mins_averages','o2a_zoneid_last5mins_averages','o2as_avg_Rid_hour_dow_30d','o2as_avg_Rz_hour_dow_30d','o2as_avg_Rz_hour_dow_9d','o2as_avg_Rz_hour_dow_lag','rest_zone_rain_mode__2H','o2as_avg_Rid_hour_3D','o2as_avg_Rz_dow_30D','o2as_avg_Rz_7D','o2as_avg_Rh_dow_7D','o2as_avg_Rh_7D','o2as_avg_Rid_dow_hour_7D']
features_o2d_regressor = ['bill_amount','banner_factor','hour','restaurant_customer_distance','o2as_avg_res_1h','o2a_zoneid_last15mins_averages','o2a_zoneid_last5mins_averages','o2as_avg_Rid_hour_dow_30d','o2as_avg_Rz_hour_dow_30d','o2as_avg_Rz_hour_dow_9d','o2as_avg_Rz_hour_dow_lag','o2as_avg_Rid_hour_3D','o2as_avg_Rz_dow_30D','o2as_avg_Rz_7D','o2as_avg_Rh_dow_7D','o2as_avg_Rh_7D','o2as_avg_Rid_dow_hour_7D','rest_zone_rain_mode__2H','cust_zone_rain_mode__2H','fm_avg_Rz_1H','fm_avg_Rid_1Day','fm_avg_Rz_dow_hour_7D','fm_avg_Rz_3H','fm_avg_Rz_dow_hour_30D','fm_avg_Rz_3D','fm_avg_Rz_1D','fm_avg_Rh_1D','fm_avg_Rh_hour_3D','fm_avg_Rh_dow_hour_7D','ar2p_avg_res_hour_20d','ar2p_avg_res_20d','ar2p_avg_res_1h','order_item_count','order_total_quantity','restaurant_active_orders','restaurant_banner_factor','items_past','o2p_value_res_lag1','item_o2p_lag','item_o2p_avg','item_slot_o2p_avg','item_slot_o2p_avg_1D','o2p_avg_1H','avg_item','high_item','rare_item','lag_delta_I','lag_delta_II','item_lag_delta_I','rest_placed_order_count__1h','restaurant_delta','o2mf_value_res_lag1','o2mf_avg_1H','p2d_avg_Rz_3D','p2d_avg_Rz_1H','p2d_avg_Cz_3D','p2d_avg_Rid_20D','p2d_avg_Rid_1H','p2d_avg_Cz_1H','p2d_avg_Rid_3D','r2d_avg_Cg_7D','p2d_avg_Ch_20D','p2d_sd_Ch_20D','embedding_hour_LM_0','embedding_hour_LM_1','embedding_hour_LM_2','embedding_hour_LM_3','embedding_hour_LM_4','embedding_city_LM_0','embedding_city_LM_1','embedding_city_LM_2','embedding_city_LM_3','embedding_city_LM_4','embedding_Rz_LM_0','embedding_Rz_LM_1','embedding_Rz_LM_2','embedding_Rz_LM_3','embedding_Rz_LM_4','embedding_Rz_LM_5','embedding_Rz_LM_6','embedding_Rz_LM_7','embedding_Rz_LM_8','embedding_Rz_LM_9','embedding_Rz_LM_10','embedding_Rz_LM_11','embedding_Rz_LM_12','embedding_Rz_LM_13','embedding_Rz_LM_14','embedding_Rz_LM_15','embedding_Rz_LM_16','embedding_Rz_LM_17','embedding_Rz_LM_18','embedding_Rz_LM_19','embedding_Rz_LM_20','embedding_Rz_LM_21','embedding_Rz_LM_22','embedding_Rz_LM_23','embedding_Rz_LM_24','embedding_hour_O2P_0','embedding_hour_O2P_1','embedding_hour_O2P_2','embedding_hour_O2P_3','embedding_hour_O2P_4','embedding_week_O2P_0','embedding_week_O2P_1','embedding_week_O2P_2','embedding_restaurant_O2P_0','embedding_restaurant_O2P_1','embedding_restaurant_O2P_2','embedding_restaurant_O2P_3','embedding_restaurant_O2P_4','embedding_restaurant_O2P_5','embedding_restaurant_O2P_6','embedding_restaurant_O2P_7','embedding_restaurant_O2P_8','embedding_restaurant_O2P_9','embedding_city_O2P_0','embedding_city_O2P_1','embedding_city_O2P_2','embedding_city_O2P_3','embedding_city_O2P_4']
features_o2p_regressor = ['bill_amount','banner_factor','hour','weekday','order_item_count','order_total_quantity','restaurant_active_orders','restaurant_banner_factor','items_past','o2p_value_res_lag1','item_o2p_lag','item_o2p_avg','item_slot_o2p_avg','item_slot_o2p_avg_1D','o2p_avg_1H','avg_item','high_item','rare_item','lag_delta_I','lag_delta_II','item_lag_delta_I','rest_placed_order_count__1h','restaurant_delta','o2mf_value_res_lag1','o2mf_avg_1H','ar2p_avg_res_hour_20d','ar2p_avg_res_20d','ar2p_avg_res_1h','rest_zone_rain_mode__2H','embedding_hour_O2P_0','embedding_hour_O2P_1','embedding_hour_O2P_2','embedding_hour_O2P_3','embedding_hour_O2P_4','embedding_week_O2P_0','embedding_week_O2P_1','embedding_week_O2P_2','embedding_restaurant_O2P_0','embedding_restaurant_O2P_1','embedding_restaurant_O2P_2','embedding_restaurant_O2P_3','embedding_restaurant_O2P_4','embedding_restaurant_O2P_5','embedding_restaurant_O2P_6','embedding_restaurant_O2P_7','embedding_restaurant_O2P_8','embedding_restaurant_O2P_9','embedding_city_O2P_0','embedding_city_O2P_1','embedding_city_O2P_2','embedding_city_O2P_3','embedding_city_O2P_4']
features_wt_regressor = ['banner_factor','weekday','hour','ar2p_avg_res_hour_20d','ar2p_avg_res_20d','ar2p_avg_res_1h','rest_zone_rain_mode__2H']

serving_features = ['item_ids','o2p_distinct_res_item_1h','o2p_avgitems_res_1Mn','o2p_highitems_res_1Mn','o2p_rareitems_res_1Mn','o2p_avg_slot_res_item_20d','o2p_avg_res_item_20d','o2p_value_res_item_lag1','o2p_avg_item_slot_1d','rest_placed_order_count__1h','rest_picked_up_order_count__1h','o2p_value_res_lag1','o2p_res_avg_1w','o2p_avg_1H','bill_amount','banner_factor','hour','restaurant_customer_distance','o2as_avg_res_1h','o2a_zoneid_last15mins_averages','o2a_zoneid_last5mins_averages','o2as_avg_Rid_hour_dow_30d','o2as_avg_Rz_hour_dow_9d','o2as_avg_Rz_hour_dow_30d','o2as_avg_Rz_hour_dow_lag','o2as_avg_Rid_hour_3D','o2as_avg_Rz_dow_30D','o2as_avg_Rz_7D','o2as_avg_Rh_dow_7D','o2as_avg_Rh_7D','o2as_avg_Rid_dow_hour_7D','rest_zone_rain_mode__2H','cust_zone_rain_mode__2H','fm_avg_Rz_1H','fm_avg_Rid_1Day','fm_avg_Rz_dow_hour_7D','fm_avg_Rz_3H','fm_avg_Rz_dow_hour_30D','fm_avg_Rz_3D','fm_avg_Rz_1D','fm_avg_Rh_1D','fm_avg_Rh_hour_3D','fm_avg_Rh_dow_hour_7D','ar2p_avg_res_hour_20d','ar2p_avg_res_20d','ar2p_avg_res_1h','order_item_count','order_total_quantity','o2mf_value_res_lag1','o2mf_avg_1H','p2d_avg_Rz_3D','p2d_avg_Rz_1H','p2d_avg_Cz_3D','p2d_avg_Rid_20D','p2d_avg_Rid_1H','p2d_avg_Cz_1H','p2d_avg_Rid_3D','r2d_avg_Cg_7D','p2d_avg_Ch_20D','p2d_sd_Ch_20D','embedding_hour_LM_0','embedding_hour_LM_1','embedding_hour_LM_2','embedding_hour_LM_3','embedding_hour_LM_4','embedding_city_LM_0','embedding_city_LM_1','embedding_city_LM_2','embedding_city_LM_3','embedding_city_LM_4','embedding_Rz_LM_0','embedding_Rz_LM_1','embedding_Rz_LM_2','embedding_Rz_LM_3','embedding_Rz_LM_4','embedding_Rz_LM_5','embedding_Rz_LM_6','embedding_Rz_LM_7','embedding_Rz_LM_8','embedding_Rz_LM_9','embedding_Rz_LM_10','embedding_Rz_LM_11','embedding_Rz_LM_12','embedding_Rz_LM_13','embedding_Rz_LM_14','embedding_Rz_LM_15','embedding_Rz_LM_16','embedding_Rz_LM_17','embedding_Rz_LM_18','embedding_Rz_LM_19','embedding_Rz_LM_20','embedding_Rz_LM_21','embedding_Rz_LM_22','embedding_Rz_LM_23','embedding_Rz_LM_24','embedding_hour_O2P_0','embedding_hour_O2P_1','embedding_hour_O2P_2','embedding_hour_O2P_3','embedding_hour_O2P_4','embedding_week_O2P_0','embedding_week_O2P_1','embedding_week_O2P_2','embedding_restaurant_O2P_0','embedding_restaurant_O2P_1','embedding_restaurant_O2P_2','embedding_restaurant_O2P_3','embedding_restaurant_O2P_4','embedding_restaurant_O2P_5','embedding_restaurant_O2P_6','embedding_restaurant_O2P_7','embedding_restaurant_O2P_8','embedding_restaurant_O2P_9','embedding_city_O2P_0','embedding_city_O2P_1','embedding_city_O2P_2','embedding_city_O2P_3','embedding_city_O2P_4','fm_avg_Rid_hour_30D','embedding_dayofweek_LM_2','embedding_dayofweek_LM_0','embedding_dayofweek_LM_1','weekday']


### Transformers

In [ ]:
def transform_fn(features):
    
    trans_features = features.copy()

    trans_features = (BinaryTensorAggTransformer(BinaryTensorOp.Intersection, BinaryTensorAggOp.Fraction, 
                                                 trans_features, cast=tf.float64).
               setInputA("o2p_distinct_res_item_1h").
               setInputB("item_ids").
               setOutputCol("items_past"))
    
    trans_features = (BinaryTensorAggTransformer(BinaryTensorOp.Intersection, BinaryTensorAggOp.Fraction,
                                           trans_features, cast=tf.float64).
               setInputA("o2p_avgitems_res_1Mn").
               setInputB("item_ids").
               setOutputCol("avg_item"))
    
    trans_features = (BinaryTensorAggTransformer(BinaryTensorOp.Intersection, BinaryTensorAggOp.Fraction,
                                           trans_features, cast=tf.float64).
               setInputA("o2p_highitems_res_1Mn").
               setInputB("item_ids").
               setOutputCol("high_item"))
    
    trans_features = (BinaryTensorAggTransformer(BinaryTensorOp.Intersection, BinaryTensorAggOp.Fraction,
                                           trans_features, cast=tf.float64).
               setInputA("o2p_rareitems_res_1Mn").
               setInputB("item_ids").
               setOutputCol("rare_item"))
    
    # insert routing as well, incase of production where there might be default fetches.
    # this max will give me random shit. Use router and length on top of this
    trans_features = (ArrayExtractorTransformer(ArrayExtractorOp.Max, trans_features, cast=tf.float64).
               setInputCol("o2p_avg_slot_res_item_20d").
               setOutputCol("item_slot_o2p_avg"))
    
    trans_features = (ArrayExtractorTransformer(ArrayExtractorOp.Max, trans_features, cast=tf.float64).
               setInputCol("o2p_avg_res_item_20d").
               setOutputCol("item_o2p_avg"))
    
    trans_features = (ArrayExtractorTransformer(ArrayExtractorOp.Max, trans_features, cast=tf.float64).
               setInputCol("o2p_value_res_item_lag1").
               setOutputCol("item_o2p_lag"))
    
    trans_features = (ArrayExtractorTransformer(ArrayExtractorOp.Max, trans_features, cast=tf.float64).
               setInputCol("o2p_avg_item_slot_1d").
               setOutputCol("item_slot_o2p_avg_1D"))
    
    trans_features = (MathBinaryTransformer(BinaryOperation.Subtract, trans_features).
               setInputA("rest_placed_order_count__1h").
               setInputB("rest_picked_up_order_count__1h").
               setOutputCol("restaurant_active_orders"))
    
    trans_features = (MathBinaryTransformer(BinaryOperation.Add, trans_features, ib=tf.constant(1, tf.float64)).
               setInputA("rest_picked_up_order_count__1h").
               setOutputCol("rest_picked_up_order_count__1h_adjusted"))
    
    trans_features = (MathBinaryTransformer(BinaryOperation.Divide, trans_features).
               setInputA("rest_placed_order_count__1h").
               setInputB("rest_picked_up_order_count__1h_adjusted").
               setOutputCol("restaurant_banner_factor"))
    
    trans_features = (MathBinaryTransformer(BinaryOperation.Subtract, trans_features).
               setInputA("o2p_value_res_lag1").
               setInputB("o2p_res_avg_1w").
               setOutputCol("lag_delta_I"))
    
    trans_features = (MathBinaryTransformer(BinaryOperation.Subtract, trans_features).
               setInputA("item_o2p_lag").
               setInputB("item_o2p_avg").
               setOutputCol("item_lag_delta_I"))
    
    trans_features = (MathBinaryTransformer(BinaryOperation.Subtract, trans_features).
               setInputA("o2p_avg_1H").
               setInputB("o2p_res_avg_1w").
               setOutputCol("lag_delta_II"))
    
    trans_features = (MathBinaryTransformer(BinaryOperation.Add, trans_features,  ib=tf.constant(1, tf.float64)).
               setInputA("rest_placed_order_count__1h").
               setOutputCol("rest_placed_order_count__1h_adjusted"))
    
    trans_features = (MathBinaryTransformer(BinaryOperation.Subtract, trans_features, ib=tf.constant(16, tf.float64)).
               setInputA("o2p_avg_1H").
               setOutputCol("restaurant_delta"))
    
    # sending entire features back, not shortlising because it happens on model level.
    # each model subsets whichever features it needs for it to run
    return trans_features

### Serving input_fn

In [ ]:
def serving_input_receiver_fn():
    
    receiver_tensors = {}
    
    for feat_name in serving_features:
        receiver_tensors[feat_name] = tf.compat.v1.placeholder(tf.float64, [None])
        
    features = receiver_tensors
    return tf.estimator.export.ServingInputReceiver(receiver_tensors=receiver_tensors,
                                                    features=transform_fn(features))

### Define RunConfig

In [ ]:
# this defines at what configuration does training run
rc = tf.estimator.RunConfig(save_checkpoints_secs = SAVE_CHECKPOINT_SECS,
                           model_dir = MODEL_DIR,
                           save_summary_steps = SAVE_SUMMARY_STEPS,
                           keep_checkpoint_max = KEEP_CHECKPOINT_MAX)

### Create estimators

In [ ]:
o2a_estimator = tf.estimator.Estimator(
    model_fn=o2a_model_fn(features_o2d_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                         LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

fm_estimator = tf.estimator.Estimator(
    model_fn=fm_model_fn(features_fm_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                        LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

wt_estimator = tf.estimator.Estimator(
    model_fn=wt_model_fn(features_wt_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                        LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

o2p_estimator = tf.estimator.Estimator(
    model_fn=o2p_model_fn(features_o2p_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                        LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

lm_estimator = tf.estimator.Estimator(
    model_fn=lm_model_fn(features_lm_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                        LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

o2d_estimator = tf.estimator.Estimator(
    model_fn=o2d_model_fn(features_o2d_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                        LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

o2d_beef_estimator = tf.estimator.Estimator(
    model_fn=o2d_beef_model_fn(features_o2d_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                        LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

### Create combined estimator

In [ ]:
estimator = CombinedEstimator([o2a_estimator, fm_estimator, wt_estimator, o2p_estimator, 
                               lm_estimator, o2d_estimator, o2d_beef_estimator],
                              ['O2As','FM','Ar2P','O2P','P2D','O2D','O2D_beef'], 
                              MODEL_DIR)

### Create SavedModel

In [ ]:
# checkpoint_path - if None, the latest checkpoint in model_dir is used
# specific checkpoint of which export-model has to be created (the most trained one)
# checkpoint_path = "o2d/model.ckpt-15847321"

estimator.export_saved_model(MODEL_DIR, 
                             serving_input_receiver_fn, 
                             checkpoint_path = None)

### Signature of the model

In [ ]:
!saved_model_cli show --dir o2d/1584003035 --all